In [1]:
import numpy as np
from keras.preprocessing import sequence
from tqdm import tqdm
import preprocessing as preproc
from utils import *

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# save_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/"

# train_path = save_path + "train/"
# val_path = save_path + "val/"

# images_vgg_4096_folder = "images_vgg_4096/"
# captions_folder = "captions/"
# indexed_captions_folder = "indexed-captions/"
# batch_folder = "batches/"
# indexed_future_words_folder = "indexed-future-words/"
# misc_folder = "misc/"
# general_datastruct_folder = "general-datastruct/"

# Load Data

In [26]:
path = app_3_length_15_data_path
base_path = path + train_folder

In [27]:
NR_TRAIN_INSTANCES = 82000
BATCH_SIZE = 2048
MAX_CAPTION_LEN = 15

In [28]:
images_precomputed_vgg_features = preproc.read_serialized_np_arr(base_path+images_vgg_4096_folder+'vgg_features.bc'
                                                                  ,nr_instances = NR_TRAIN_INSTANCES )

print(images_precomputed_vgg_features.shape)

(62735, 4096)


In [29]:
raw_captions = preproc.get_truncated_captions_from_batch(base_path + captions_folder, batch_nr = 0,
                                                           nr_instances = NR_TRAIN_INSTANCES )

print(len(raw_captions))

62735


In [30]:
unique_words = preproc.load_obj(path+general_datastruct_folder+"unique_words")
word2index = preproc.load_obj(path+general_datastruct_folder+"word2index")
index2word = preproc.load_obj(path+general_datastruct_folder+"index2word")
VOCAB_SIZE = len(unique_words)
print(len(unique_words))

7275


# Save img vgg features batch

In [32]:
for index in tqdm(range(len(images_precomputed_vgg_features) / BATCH_SIZE)):
    img_vgg_feature_list = []
    
    for elem_in_batch in range(BATCH_SIZE):
        img_vgg_feature = np.expand_dims(images_precomputed_vgg_features[index*BATCH_SIZE + elem_in_batch],axis=0)
        img_vgg_feature_list.append(img_vgg_feature)
        
    img_vgg_batch = np.vstack(img_vgg_feature_list)
    save_array(base_path + batch_folder + images_vgg_4096_folder + 'img_vgg_feature_' + str(format(index, "06"))
               + '_' + '.bc',img_vgg_batch)

100%|██████████| 30/30 [00:03<00:00,  9.07it/s]


# Save raw captions batch

In [33]:
for index in tqdm(range(len(raw_captions)/BATCH_SIZE)):
    caption_list = []
    
    for elem_in_batch in range(BATCH_SIZE):
        caption_list.append(raw_captions[index*BATCH_SIZE + elem_in_batch])
     
    captions_batch = np.vstack(caption_list)
    
    save_array(base_path+batch_folder+captions_folder+'caption_'+str(format(index, "06"))+'_'+'.bc',
               captions_batch)

100%|██████████| 30/30 [00:00<00:00, 106.84it/s]


# Save indexed captions batch

In [34]:
indexed_captions_folder = "indexed-captions/"
indexed_future_words_folder = "indexed-future-words/"

In [36]:
indexed_captions = []
for index,raw_caption in tqdm(enumerate(raw_captions)):
    indexed_caption = [word2index[caption_word] for caption_word in raw_caption.split()]
    indexed_caption = sequence.pad_sequences([indexed_caption], maxlen=MAX_CAPTION_LEN,padding='post')
    indexed_np_arr = np.asarray(np.squeeze(indexed_caption))
    
    indexed_captions.append(indexed_np_arr)
    
    
for index in tqdm(range(len(indexed_captions)/BATCH_SIZE)):
    
    indexed_caption_list = []
    
    for elem_in_batch in range(BATCH_SIZE):
        indexed_caption = np.expand_dims(indexed_captions[index*BATCH_SIZE + elem_in_batch],axis=0)
        indexed_caption_list.append(indexed_caption)
        
    indexed_captions_batch = np.vstack(indexed_caption_list)
    
    save_array(base_path + batch_folder + indexed_captions_folder + 'indexed_caption_'
               + str(format(index, "06")) + '_' + '.bc',
               indexed_captions_batch)

62735it [00:01, 51277.93it/s]
100%|██████████| 30/30 [00:00<00:00, 151.39it/s]


# Save future words captions batch

In [ ]:
for index in tqdm((range(len(indexed_captions)/BATCH_SIZE))):

    indexed_future_word_list = []
        
    for elem_in_batch in range(BATCH_SIZE):

        caption_indexed = indexed_captions[index*BATCH_SIZE + elem_in_batch]
            
        enhanced_caption_indexed = np.append(caption_indexed,[word2index["END"]]) #hacky
        word_2_next_word = []

        for i in xrange(0,len(caption_indexed)):
            caption_word_index = enhanced_caption_indexed[i]
            future_word_index = enhanced_caption_indexed[i+1]
            future_indexes = np.zeros(VOCAB_SIZE)
            future_indexes[future_word_index] = 1

            word_2_next_word.append(future_indexes)

        words_2_next_word = np.vstack(word_2_next_word)
        words_2_next_word = np.expand_dims(words_2_next_word,axis=0)
        
        indexed_future_word_list.append(words_2_next_word)
    
    indexed_future_words_batch = np.vstack(indexed_future_word_list)
    
    save_array(base_path+batch_folder+indexed_future_words_folder+'indexed_future_word_'+str(format(index, "06"))+'_'+'.bc', 
               indexed_future_words_batch)
    


  7%|▋         | 2/30 [00:07<01:42,  3.67s/it]